In [103]:
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver import ActionChains 
from time import sleep
import time

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

In [114]:
options = Options()
ua = UserAgent()
userAgent = ua.random
options.add_argument('--start-fullscreen')  # 전체화면
options.add_argument(f'user-agent={userAgent}') # useragent값
options.add_argument('--disable-blink-features=AutomationControlled')   # 봇 피하기
driver = webdriver.Chrome(chrome_options=options, executable_path=r'./chromedriver')    # 크롬주소
sleep(3)

In [115]:
driver.get("https://www.nike.com/kr/ko_kr/w/new/fw/xc/new-mens-shoes")
sleep(3)


In [116]:
# 스크롤 할 페이지 횟수
n=int(input())

In [117]:
# 나이키 페이지 스크롤 다운
for i in range(n):
   
    end_tag=driver.find_element_by_xpath('/html/body/section/section/section/article/div/div[2]/div')
    
    ActionChains(driver).move_to_element(end_tag).perform()
    driver.execute_script("window.scrollTo(0, window.scrollY + 500);")
    time.sleep(2)


In [ ]:
# 상품의 링크를 모두 들고와 item_url에 저장
items=driver.find_elements_by_css_selector(".a-product-image-link")
item_url=[i.get_attribute('href') for i in items]

In [ ]:
len(item_url)

In [ ]:
result_list=[]

for shoe_url in item_url:
    
    driver.get(shoe_url)
    sleep(2)
    try:
        # 신발 카테고리
        shoe_cat=driver.find_element_by_xpath("/html/body/section/section/section/article/article[2]/div/div[4]/div/div[1]/div/div[1]/div/span").text
        # 신발 이름
        shoe_name=driver.find_element_by_xpath("/html/body/section/section/section/article/article[2]/div/div[4]/div/div[1]/h1/span").text
        # 신발 가격
        shoe_price=driver.find_element_by_xpath('/html/body/section/section/section/article/article[2]/div/div[4]/div/div[1]/div/div[2]/span/strong').text
        
        #리뷰 더보기
        more_review=driver.find_element_by_xpath('/html/body/section/section/section/article/article[2]/div/div[4]/div/div[4]/h2[1]')
        # 리뷰 수
        num_of_reviews=more_review.text.split('(')[1].split(')')[0]
        more_review.click()
        sleep(3)
        driver.find_element_by_xpath('//*[@id="detail-review"]/div/div[2]/a').click()
        sleep(2)

        if int(num_of_reviews)>2000:
            n=2000
        else:
            n=int(num_of_reviews)
        
        try:
            review_more=driver.find_element_by_id('load-more')
            for i in range(n//20):
                review_more.click()
                sleep(2)

            review_ul=driver.find_element_by_id('review-list')
            review_list=review_ul.find_elements_by_tag_name('li')
            sleep(2)

            for i in review_list:
                review=i.find_element_by_class_name('read-comment').text
                result_list.append({
                    'ShoeName':shoe_name,
                    'ShoeCat':shoe_cat,
                    'ShoePrice':shoe_price,
                    'Review':review
                })

        except:
            review_ul=driver.find_element_by_id('review-list')
            review_list=review_ul.find_elements_by_tag_name('li')
            sleep(2)

            for i in review_list:
                review=i.find_element_by_class_name('read-comment').text
                result_list.append({
                    'ShoeName':shoe_name,
                    'ShoeCat':shoe_cat,
                    'ShoePrice':shoe_price,
                    'Review':review
                })
    
    except:
        pass

In [ ]:
df_url=pd.Series(item_url)
df_url.to_csv('u.csv')

In [ ]:
df=pd.DataFrame(result_list)
df.to_csv('nike.csv')